# Setup

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch as t
import torch.nn.functional as F

In [2]:
model_name = 'gpt2'
device = 'cuda:0' if t.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=t.float16).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# Functions

In [3]:
def actvs_to_logits(hidden_states):
    """
    outputs.hidden_states is a tuple for every layer
    each tuple member is an actvs tensor of size (batch_size, seq_len, d_model)
    loop thru tuple to get actv for each layer
    """
    layer_logits_list = []  # logits for each layer hidden state output actvs
    for i, h in enumerate(hidden_states):
        h_last_tok = h[:, -1, :]
        if i == len(hidden_states) - 1:
            h_last_tok = model.transformer.ln_f(h_last_tok)  # apply layer norm as not in last
        logits = t.einsum('ab,cb->ac', model.lm_head.weight, h_last_tok)
        layer_logits_list.append(logits)
    return layer_logits_list

def get_logits(input_text):
    token_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    outputs = model(token_ids, output_hidden_states=True)
    logits = actvs_to_logits(outputs.hidden_states)
    logits = t.stack(logits).squeeze(-1)
    return logits

In [4]:
def get_decoded_indiv_toks(layer_logits, k=10):
    """
    i is the layer (from before to last).
    layer_logits[i] are the scores for each token in vocab dim for the ith unembedded layer
    j is the top 5
    """
    output_list = []
    for i, layer in enumerate(layer_logits):
        top_5_at_layer = []
        sorted_token_ids = F.softmax(layer_logits[i],dim=-1).argsort(descending=True)
        for j in range(5):  # loop to separate them in a list, rather than concat into one str
            top_5_at_layer.append(tokenizer.decode(sorted_token_ids[j]))
        output_list.append( top_5_at_layer )
    return output_list

# Test one samp each type

In [5]:
prompts = ["1 2 3 4", "one two three four", "January February March April"]
for test_text in prompts:
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' 4', ' 3', ' 5', ' 2', ' 6']
0 [' the', '-', ' a', ',', ' "']
1 ['th', '.', '-', ' the', ',']
2 ['.', ',', 'th', '-', ' the']
3 ['.', ' -', 'th', '.', ' X']
4 [' -', '.', ' a', ' 4', 'th']
5 [' -', ' 3', '.', ' 4', ' 5']
6 [' 4', ' 3', ' 5', ' 1', '.']
7 [' 4', ' 5', ' 3', ' 6', '!']
8 [' 4', ' 5', ' 3', '!', '"']
9 [' 5', '!', '"', '#', '$']
10 [' 5', '-', '!', '"', '#']
11 [' 5', '\n', ',', '.', ' (']


-1 [' four', ' three', ' five', 'four', 'three']
0 [' the', '-', ' and', ' a', ' "']
1 ['teen', '-', 'teenth', ' "', ' three']
2 ['teen', '-', ' hundred', 'teenth', 'some']
3 ['teen', '-', ' hundred', ' times', 'teenth']
4 ['teen', '-', ' of', ' times', 'teenth']
5 ['-', ',', ' of', ' and', '.']
6 ['-', ',', '.', ' and', ' of']
7 ['-', '\n', ',', ' and', ' 4']
8 [' 4', '-', '\n', ' 3', ' 5']
9 [' five', ' 5', '!', '"', '#']
10 [',', '-', '.', '!', '"']
11 [',', '.', '\n', ' and', ' the']


-1 [' April', 'April', ' June', ' February', 'June']
0 [',', ' the', '-', '.', ' and']
1 ['

# pure seq prompts

In [ ]:
# def generate_prompts_list(x ,y):
#     prompts_list = []
#     for i in range(x, y):
#         prompt_dict = {
#             'corr': str(i+4),
#             'incorr': str(i+3),
#             'text': f"{i} {i+1} {i+2} {i+3}"
#         }
#         prompts_list.append(prompt_dict)
#     return prompts_list

# prompts_list = generate_prompts_list(1, 9)

In [ ]:
# for pd in prompts_list:
#     test_text = pd['text']
#     layer_logits = get_logits(test_text)
#     tok_logit_lens = get_decoded_indiv_toks(layer_logits)
#     for i, tokouts in enumerate(tok_logit_lens):
#         print(i-1, tokouts)
#     print('\n')

-1 [' 4', ' livest', ' mathemat', ' myster', ' horizont']
0 ['th', 'teenth', 'x', ' 4', ' 3']
1 ['th', 'teenth', '54', 'x', '39']
2 ['th', 'x', '34', '54', 'GHz']
3 ['34', '54', 'th', 'GHz', '86']
4 ['ts', 'ms', '34', 'th', 'ths']
5 ['ts', ' 4', ' -', ' 3', ' 5']
6 [' 4', '★', ' 3', ' 5', ' 6']
7 [' 4', ' 5', ' 6', ' 3', ' 0']
8 [' 4', ' 5', ' 3', ' 6', ' 1']
9 [' 5', ' 4', ' 6', ' 3', ' 7']
10 [' 5', ' 6', ' 4', ' 3', ' 7']
11 [' 5', ' 4', ' 1', ' 6', ' 3']


-1 [' livest', ' 5', ' destro', 'theless', ' mathemat']
0 ['th', ' 5', ' 3', 'x', ' times']
1 ['th', 'x', '54', '45', '43']
2 ['th', '34', 'x', 'min', '54']
3 ['34', 'th', '90', '54', '85']
4 ['ths', 'ts', ' -', 'min', '34']
5 [' -', 'ts', ' 3', 'ths', ' 5']
6 [' 5', ' 4', ' 3', ' 6', '★']
7 [' 5', ' 6', ' 4', ' 3', ' 9']
8 [' 5', ' 6', ' 4', ' 3', ' 7']
9 [' 6', ' 5', ' 4', ' 7', '6']
10 [' 6', ' 5', ' 4', ' 7', ' 8']
11 [' 6', ' 5', ' 4', ' 7', ' 0']


-1 [' 6', ' livest', ' destro', ' mathemat', ' challeng']
0 ['th', '36', '31

# pure seq prompts nw

In [ ]:
# def generate_prompts_list(x ,y):
#     words = [' one', ' two', ' three', ' four', ' five', ' six', ' seven', ' eight', ' nine', ' ten', ' eleven', ' twelve', ' thirteen', ' fourteen', ' fifteen', ' sixteen', ' seventeen', ' eighteen', ' nineteen', ' twenty']
#     prompts_list = []
#     for i in range(x, y):
#         prompt_dict = {
#             'S1': words[i],
#             'S2': words[i+1],
#             'S3': words[i+2],
#             'S4': words[i+3],
#             'corr': words[i+4],
#             'incorr': words[i+3],  # this is arbitrary
#             'text': f"{words[i]}{words[i+1]}{words[i+2]}{words[i+3]}"
#         }
#         prompts_list.append(prompt_dict)
#     return prompts_list

# prompts_list = generate_prompts_list(0, 8)

In [ ]:
# for pd in prompts_list:
#     test_text = pd['text']
#     layer_logits = get_logits(test_text)
#     tok_logit_lens = get_decoded_indiv_toks(layer_logits)
#     for i, tokouts in enumerate(tok_logit_lens):
#         print(i-1, tokouts)
#     print('\n')

# pure seq prompts months

In [ ]:
# def generate_prompts_list(x ,y):
#     words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
#     prompts_list = []
#     for i in range(x, y):
#         prompt_dict = {
#             'S1': words[i],
#             'S2': words[i+1],
#             'S3': words[i+2],
#             'S4': words[i+3],
#             'corr': words[i+4],
#             'incorr': words[i+3],  # this is arbitrary
#             'text': f"{words[i]}{words[i+1]}{words[i+2]}{words[i+3]}"
#         }
#         prompts_list.append(prompt_dict)
#     return prompts_list

# prompts_list = generate_prompts_list(0, 8)

In [ ]:
# for pd in prompts_list:
#     test_text = pd['text']
#     layer_logits = get_logits(test_text)
#     tok_logit_lens = get_decoded_indiv_toks(layer_logits)
#     for i, tokouts in enumerate(tok_logit_lens):
#         print(i-1, tokouts)
#     print('\n')

-1 ['April', 'June', 'February', 'October', 'September']
0 [' 2015', ' 2014', ' May', ' Aug', ' 2013']
1 [' 2015', ' 2014', ' 2017', ' 2013', ' 2016']
2 [' 2014', ' 2015', ' 2013', ' 29', ' 2017']
3 [' 2015', ' 2014', ' 29', ' 2017', ' 28']
4 [' 2014', ' 2015', ' 29', ' 2017', ' 30']
5 [' 29', ' 30', ' 2014', ' 28', '/']
6 [' 2014', ' 29', '/', ' 30', ' 2015']
7 ['April', 'December', 'June', 'September', 'October']
8 ['April', 'June', 'March', 'September', 'October']
9 ['May', 'April', 'June', 'Apr', 'October']
10 ['May', 'June', 'April', 'July', 'October']
11 ['May', 'June', 'July', 'April', 'October']


-1 ['May', ' livest', 'June', 'April', 'Apr']
0 [' May', 'May', 'flower', ' 2015', 'Aug']
1 [' 2015', ' 2014', ' May', ' 2013', 'flower']
2 [' 2014', ' 2015', 'flower', ' 2011', ' 2013']
3 [' 2015', 'hem', ' 2014', 'nard', ' 31']
4 ['weather', 'hem', ' 2014', 'yy', ' 2015']
5 ['weather', '/', ']', ' 29', 'way']
6 ['flower', 'May', 'June', 'December', 'hem']
7 ['hem', 'flower', 'May', 

# nw- 1536 prompts, among words

In [ ]:
import pickle

task = "nw"
prompts_list = []

temps = ['done', 'lost', 'names']

for i in temps:
    file_name = f'/content/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:512] #768 512

len(prompts_list)

Van done in one. Hat done in two. Ring done in three. Desk done in four. Sun done in
Oil lost in one. Apple lost in two. Tree lost in three. Snow lost in four. Apple lost in
Marcus born in one. Victoria born in two. George born in three. Brandon born in four. Jamie born in


1536

In [ ]:
num_words = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
                 "eleven", "twelve"]

anomolies = []
num_corr = 0
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)

    """
    Check if the 8th layer's predicted token is the sequence member just "one before"
    the correct next sequence member output found in the ninth layer

    Use `try` because when indexing, the output may not be a seq member of the right type!
    """
    try:
        a = num_words.index(tok_logit_lens[9][0].replace(' ', ''))
        b= num_words.index(tok_logit_lens[10][0].replace(' ', ''))
        if int(a) < int(b):
            if tok_logit_lens[10][0] == pd['corr']:
                num_corr += 1
            else:
                anomolies.append(pd)
        else:
                anomolies.append(pd)
    except:
        anomolies.append(pd)
    # for i, tokouts in enumerate(tok_logit_lens):
    #     print(i-1, tokouts)
    # print('\n')

In [ ]:
num_corr

1396

Do a quick scan of what prompts are anomolies in which 8th layer output is not just "the seq member one before" the 9th layer output.

In [ ]:
for pd in anomolies[:2]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' vain', ' particular']
2 [' order', ' the', ' vain', ' front', ' conjunction']
3 [' order', ' the', ' vain', ' conjunction', ' accordance']
4 [' order', ' the', ' relation', ' vain', ' conjunction']
5 [' order', ' vain', ' lieu', ' conjunction', ' the']
6 [' order', ' three', ' conjunction', ' lieu', ' a']
7 [' three', ' five', ' seven', ' nine', ' order']
8 [' seven', ' six', ' five', ' four', ' eight']
9 [' seven', ' eight', ' six', ' nine', ' seventh']
10 [' seven', ' eight', ' nine', ' six', ' nineteen']
11 [' seven', ' eight', ' six', ' nine', ' ten']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' front']
1 [' order', ' the', ' conjunction', ' vain', ' particular']
2 [' order', ' the', ' vain', ' conjunction', ' front']
3 [' order', ' vain', ' the', ' accordance',

In [ ]:
for pd in anomolies[-2:]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' front']
1 [' the', ' order', ' a', ' conjunction', ' particular']
2 [' the', ' order', ' a', ' conjunction', ' front']
3 [' the', ' order', ' accordance', ' conjunction', ' a']
4 [' the', ' front', ' order', ' conjunction', ' his']
5 [' order', ' 18', ' 17', ' front', ' conjunction']
6 [' front', ' 10', ' order', ' 3', ' three']
7 [' nine', ' seven', ' 9', ' eight', ' 12']
8 [' nine', ' eight', ' 10', ' ten', ' seven']
9 [' 10', ' ten', ' nine', ' twelve', ' tenth']
10 [' ten', ' twelve', ' eleven', ' 10', ' thirteen']
11 [' ten', ' 10', ' twelve', ' eleven', ' fifteen']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' the', ' order', ' particular', ' conjunction', ' a']
2 [' the', ' order', ' a', ' conjunction', ' front']
3 [' the', ' order', ' accordance', ' his', ' conjunction']
4 [' the', ' order', 

# months 1536

In [ ]:
import pickle

task = "months"
prompts_list = []

temps = ['done', 'lost', 'names']

for i in temps:
    file_name = f'/content/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:512] #768 512

len(prompts_list)

Van done in January. Hat done in February. Ring done in March. Desk done in April. Sun done in
Oil lost in January. Apple lost in February. Tree lost in March. Snow lost in April. Apple lost in
Marcus born in January. Victoria born in February. George born in March. Brandon born in April. Jamie born in


1536

In [ ]:
num_words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

anomolies = []
num_corr = 0
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)

    """
    Check if the 8th layer's predicted token is the sequence member just "one before"
    the correct next sequence member output found in the ninth layer

    Use `try` because when indexing, the output may not be a seq member of the right type!
    """
    try:
        a = num_words.index(tok_logit_lens[9][0].replace(' ', ''))
        b = num_words.index(tok_logit_lens[10][0].replace(' ', ''))
        if int(a) < int(b):
            if tok_logit_lens[10][0] == pd['corr']:
                num_corr += 1
            else:
                anomolies.append(pd)
        else:
                anomolies.append(pd)
    except:
        anomolies.append(pd)
    # for i, tokouts in enumerate(tok_logit_lens):
    #     print(i-1, tokouts)
    # print('\n')

In [ ]:
num_corr

1495

Do a quick scan of what prompts are anomolies in which 8th layer output is not just "the seq member one before" the 9th layer output.

In [ ]:
for pd in anomolies[:2]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' particular', ' vain']
2 [' order', ' the', ' conjunction', ' vain', ' accordance']
3 [' order', ' the', ' vain', ' accordance', ' conjunction']
4 [' order', ' the', ' conjunction', ' accordance', ' vain']
5 [' order', ' lieu', ' vain', ' late', ' conjunction']
6 [' order', ' lieu', ' June', ' conjunction', ' late']
7 [' June', ' September', ' May', ' March', ' July']
8 [' September', ' August', ' July', ' June', ' October']
9 [' September', ' October', ' November', ' August', ' Sept']
10 [' September', ' October', ' November', ' August', ' December']
11 [' September', ' October', ' November', ' August', ' December']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' particular', ' my']
2 [' order', ' the', ' conjunction

In [ ]:
for pd in anomolies[-2:]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' particular', ' a']
2 [' the', ' order', ' a', ' accordance', ' front']
3 [' the', ' order', ' accordance', ' a', ' his']
4 [' front', ' the', ' order', ' conjunction', ' his']
5 [' order', ' lieu', ' late', ' 1968', ' front']
6 [' lieu', ' order', ' June', ' conjunction', ' front']
7 [' June', ' September', ' August', ' July', ' May']
8 [' September', ' August', ' July', ' October', ' November']
9 [' September', ' October', ' November', ' December', ' August']
10 [' September', ' October', ' November', ' December', ' August']
11 [' September', ' October', ' November', ' December', ' January']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' particular', ' a']
2 [' order', ' the', ' a', ' accordance', ' conjunction']
3

# numerals 1536

In [ ]:
task = "digits"
prompts_list = []

temps = ['done', 'lost', 'names']

for i in temps:
    file_name = f'/content/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:512] #768 512

len(prompts_list)

Van done in 1. Hat done in 2. Ring done in 3. Desk done in 4. Sun done in
Oil lost in 1. Apple lost in 2. Tree lost in 3. Snow lost in 4. Apple lost in
Marcus born in 1. Victoria born in 2. George born in 3. Brandon born in 4. Jamie born in


1536

In [ ]:
num_corr = 0
anomolies = []
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)

    """
    Check if the 8th layer's predicted token is the sequence member just "one before"
    the correct next sequence member output found in the ninth layer

    Use `try` because when indexing, the output may not be a seq member of the right type!
    """
    try:
        a = tok_logit_lens[9][0].replace(' ', '')
        b= tok_logit_lens[10][0].replace(' ', '')
        if int(a) < int(b):
            if tok_logit_lens[10][0] == pd['corr']:
                num_corr += 1
            else:
                anomolies.append(pd)
        else:
                anomolies.append(pd)
    except:
        anomolies.append(pd)
    # for i, tokouts in enumerate(tok_logit_lens):
    #     print(i-1, tokouts)
    # print('\n')

1531

In [ ]:
num_corr

1531

Do a quick scan of what prompts are anomolies in which 8th layer output is not just "the seq member one before" the 9th layer output.

In [ ]:
for pd in anomolies[:2]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' conjunction', ' a']
1 [' order', ' the', ' conjunction', ' particular', ' a']
2 [' order', ' the', ' vain', ' front', ' conjunction']
3 [' order', ' the', ' vain', ' conjunction', ' front']
4 [' order', ' the', ' conjunction', ' vain', ' spite']
5 [' order', ' conjunction', ' 3', ' vain', ' 15']
6 [' 3', ' 5', ' 2', ' 4', ' 7']
7 [' 3', ' 5', ' 7', ' 4', ' 2']
8 [' 7', ' 6', ' 5', ' 8', ' 1']
9 [' 7', ' 8', ' 6', ' 9', '7']
10 [' 7', ' 8', ' 9', ' 6', ' 1']
11 [' 7', ' 8', ' 1', ' 9', ' 6']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' a', ' particular']
2 [' order', ' the', ' conjunction', ' vain', ' front']
3 [' order', ' the', ' conjunction', ' vain', ' accordance']
4 [' order', ' conjunction', ' the', ' relation', ' accordance']
5 [' order', ' 3', ' 5', ' 18', ' 6']
6 

In [ ]:
for pd in anomolies[-2:]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' front']
1 [' order', ' the', ' a', ' conjunction', ' front']
2 [' order', ' the', ' a', ' vain', ' accordance']
3 [' order', ' the', ' vain', ' his', ' a']
4 [' order', ' the', ' spite', ' relation', ' vain']
5 [' order', ' 3', ' 2', ' 5', ' 1']
6 [' 3', ' 2', ' 5', ' 1', ' 4']
7 [' 3', ' 2', ' 1', ' 5', ' 4']
8 [' 5', ' 4', ' 3', ' 1', ' 2']
9 [' 5', ' 6', ' 1', ' 7', '5']
10 [' 5', ' 6', ' 1', ' 7', ' 3']
11 [' 5', ' 6', ' 1', ' 7', ' 3']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' the', ' order', ' a', ' particular', ' front']
1 [' order', ' the', ' a', ' conjunction', ' particular']
2 [' order', ' the', ' a', ' accordance', ' vain']
3 [' order', ' the', ' accordance', ' relation', ' vain']
4 [' order', ' the', ' accordance', ' spite', ' relation']
5 [' order', ' 3', ' 5', ' 18', ' 15']
6 [' 3', ' 5', ' 2', ' 7', ' 4']
7 [' 3', ' 5', ' 7', ' 4', ' 6']
